In [1]:
!pip  install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19558 sha256=e78e071b252d33c1fa00d6a434775df85fc5070a36f9733491ab401b3e762514
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.0.0
    Uninstalling absl-py-1.0.0:
      Successfully uninstalled absl-py-1.0.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-est

In [1]:
import tensorflow as tf 
import numpy as np 
from matplotlib import pyplot as plt 
import cv2

In [9]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
interpreter.allocate_tensors()

**Draw Keypoints**

In [10]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped: 
        ky, kx, kp_conf = kp 
        if kp_conf > confidence_threshold: 
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

**Draw Edges**

In [11]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [12]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1,p2 = edge 
        y1, x1, c1 = shaped[p1] 
        y2, x2, c2 = shaped[p2]
        
        if(c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255),2)

In [14]:
cap = cv2.VideoCapture(0) #connects to webcam
while cap.isOpened(): #loop through every frame
    ret, frame = cap.read() 
    
    #Reshape 
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0), 256, 256)
    input_image = tf.cast(img, dtype=tf.float32)
    # Setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    # rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame) #rendering to the screen 
    
    if cv2.waitKey(10) & 0xFF==ord('q'): 
        break 
cap.release()
cv2.destroyAllWindows()

[[[[0.56255805 0.47955996 0.5829028 ]
   [0.47006997 0.54351133 0.7401744 ]
   [0.47787645 0.40527654 0.6690502 ]
   [0.45140624 0.6315846  0.66788423]
   [0.47885922 0.318961   0.6639898 ]
   [0.6737404  0.7928824  0.41774672]
   [0.7791074  0.2548792  0.69353867]
   [0.9886913  1.0204124  0.01796815]
   [0.8729602  0.1736955  0.0278863 ]
   [0.77972215 0.73909116 0.00604963]
   [0.8864813  0.1849245  0.01285452]
   [1.0061342  0.9750053  0.00824708]
   [0.99028975 0.22342417 0.00785592]
   [0.74588144 0.7644619  0.00671688]
   [0.84112495 0.21802689 0.02313253]
   [0.46211025 0.5349378  0.02451304]
   [0.86340475 0.4769287  0.00327674]]]]
[[[[0.5654483  0.47753873 0.6631031 ]
   [0.46849412 0.5402271  0.80295956]
   [0.47634968 0.40399823 0.7863858 ]
   [0.4534493  0.62519634 0.6485903 ]
   [0.4809692  0.31608304 0.6786976 ]
   [0.6729485  0.80085915 0.66105336]
   [0.7815469  0.23671837 0.7678827 ]
   [0.86017156 0.89994174 0.00621116]
   [0.87730294 0.17359215 0.03675854]
   [0.546